### Testing

In [ ]:
%pip install nest_asyncio

In [ ]:
from langsmith import wrappers, Client
from pydantic import BaseModel, Field
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = Client()
openai_client = wrappers.wrap_openai(OpenAI())

examples = [
    {
        "question": "2025년에 LangGraph 에이전트를 가장 많이 채택한 회사들은 어디인가요?",
        "answer": "Uber, LinkedIn, Replit과 같은 선도 기업들은 강력할 뿐만 아니라 신뢰성까지 갖춘 에이전트를 구축하기 위해 LangGraph를 선택하고 있습니다. :cite[3]"
    },
    {
        "question": "AppFolio는 LangGraph를 사용하면서 어떤 개선을 이루었나요?",
        "answer": "AppFolio는 부동산 관리자의 업무 시간 을 주당 10시간 이상 절감해 주는 코파일럿을 개발했고 , LangGraph는 앱 지연 시간을 줄이고 의사 결정의 정확도를 2배 높이는 데 도움을 주었습니다. :cite[4]"
    },
    {
        "question": "컨텍스트 엔지니어링은 왜 중요한가요?",
        "answer": "LangGraph에서 컨텍스트 엔지니어링(Context Engineering)이 중요한 이유는, LangGraph가 에이전트 기반의 LLM 워크플로우를 정의하고 실행하는 시스템이기 때문입니다. 이 구조에서는 각 노드(에이전트)가 이전 상태를 바탕으로 행동하게 되며, 이때 전달되는 컨텍스트가 각 에이전트의 판단과 출력을 결정짓는 핵심 요소가 됩니다. :cite[2]:cite[5]"
    },
    {
        "question": "Anthropic 팀이 컨텍스트 엔지니어링에 상당한 시간을 투자했음을 어떻게 알 수 있습니까?",
        "answer": "장거리 대화 관리.  프로덕션 에이전트는 종종 수백 턴에 걸쳐 대화에 참여하기 때문에 신중한 맥락 관리 전략이 필요합니다. 대화가 확장됨에 따라 표준 맥락 창이 부족해지면서 지능형 압축 및 메모리 메커니즘이 필요하게 됩니다. :cite[2]:cite[7]"
    },
    {
        "question": "Elastic의 LangGraph 구현은 무엇이 다른가요?",
        "answer": "Elastic은 보안 중심 자동화 기능과 Elasticsearch 기반 RAG 에이전트 구현을 위한 종합 템플릿/툴킷을 제공함으로써, 일반적인 LangGraph 사용보다 도메인 특화, 검색 최적화, 운영 강조 측면에서 확연히 구분됩니다.  :cite[3]"
    }
]


inputs = [{"question": example["question"]} for example in examples]
outputs = [{"answer": example["answer"]} for example in examples]

# LangSmith의 Dataset을 생성
dataset = client.create_dataset(
    dataset_name="langgraph-blogs-qa", description="LangGraph blogs QA."
)

# dataset에 example 데이터 추가
client.create_examples(inputs=inputs, outputs=outputs, dataset_id=dataset.id)

print(
    f"Dataset created in langsmith with ID: {dataset.id}\n Navigate to {dataset.url}.")


In [ ]:
from typing import List, TypedDict
from langchain_community.document_loaders import WebBaseLoader
from langchain.schema import Document
from langgraph.graph import END, StateGraph, START
from langchain_community.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain import hub
from langchain_openai import ChatOpenAI


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        scraped_documents: list of documents
        vectorstore: vectorstore
    """

    question: str
    scraped_documents: List[str]
    vectorstore: InMemoryVectorStore
    answer: str


def scrape_blog_posts(state) -> List[Document]:
    """
    Scrape the blog posts and create a list of documents
    """

    urls = [
        "https://blog.langchain.com/is-langgraph-used-in-production/",
        "https://blog.langchain.com/the-rise-of-context-engineering/",
        "https://blog.langchain.com/why-langgraph-platform/",
    ]

    docs = [WebBaseLoader(url).load() for url in urls]
    docs_list = [item for sublist in docs for item in sublist]

    return {"scraped_documents": docs_list}


def indexing(state):
    """
    Index the documents
    """
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=250, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(state["scraped_documents"])

    # Add to vectorDB
    vectorstore = InMemoryVectorStore.from_documents(
        documents=doc_splits,
        embedding=OpenAIEmbeddings(),
    )
    return {"vectorstore": vectorstore}


def retrieve_and_generate(state):
    """
    Retrieve documents from vectorstore and generate answer
    """
    question = state["question"]
    vectorstore = state["vectorstore"]

    retriever = vectorstore.as_retriever()

    prompt = hub.pull("rlm/rag-prompt")
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

    # fetch relevant documents
    docs = retriever.invoke(question)  # format prompt
    formatted = prompt.invoke(
        {"context": docs, "question": question})  # generate answer
    answer = llm.invoke(formatted)
    return {"answer": answer}


# Graph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve_and_generate", retrieve_and_generate)  # retrieve
workflow.add_node("scrape_blog_posts", scrape_blog_posts)  # scrape web
workflow.add_node("indexing", indexing)  # index

# Build graph
workflow.add_edge(START, "scrape_blog_posts")
workflow.add_edge("scrape_blog_posts", "indexing")
workflow.add_edge("indexing", "retrieve_and_generate")

workflow.add_edge("retrieve_and_generate", END)

# Compile
graph = workflow.compile()


In [ ]:
from typing import Optional
from langchain_openai import ChatOpenAI
from langsmith import Client, evaluate, aevaluate
from langsmith.evaluation import EvaluationResults
from pydantic import BaseModel, Field
from typing_extensions import Annotated, TypedDict
from dotenv import load_dotenv
import nest_asyncio

load_dotenv()
client = Client()

DEFAULT_DATASET_NAME = "langgraph-blogs-qa"

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

EVALUATION_PROMPT = f"""당신은 퀴즈를 채점하는 선생님입니다.

                        질문, 정답(정답), 학생 답안이 주어집니다.

                        채점 기준은 다음과 같습니다.
                        (1) 정답 대비 사실적 정확성만을 기준으로 학생 답안을 채점합니다.
                        (2) 학생 답안에 상충되는 진술이 없는지 확인합니다.
                        (3) 정답 대비 사실적 정확성을 유지하는 한, 학생 답안에 정답보다 더 많은 정보가 포함되어 있어도 괜찮습니다.

                        정확성(Correctness):
                        참은 학생의 답안이 모든 기준을 충족함을 의미합니다.
                        거짓은 학생의 답안이 모든 기준을 충족하지 못함을 의미합니다.

                        추론과 결론이 정확한지 확인하기 위해 단계별로 추론 과정을 설명하십시오."""

# LLM-as-judge output schema

class Grade(TypedDict):
    """Compare the expected and actual answers and grade the actual answer."""
    reasoning: Annotated[str, ...,
                         "실제 응답이 맞는지 아닌지에 대한 추론을 설명하세요."]
    is_correct: Annotated[bool, ...,
                          "학생의 응답이 대부분 또는 정확히 맞으면 참이고, 그렇지 않으면 거짓입니다."]


grader_llm = llm.with_structured_output(Grade)


def transform_dataset_inputs(inputs: dict) -> dict:
    """Transform LangSmith dataset inputs to match the agent's input schema before invoking the agent."""
    return inputs


def transform_agent_outputs(outputs: dict) -> dict:
    """Transform agent outputs to match the LangSmith dataset output schema."""
    return {"info": outputs["info"]}


# Evaluator function
async def evaluate_agent(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    """Evaluate if the final response is equivalent to reference response."""
    
    user = f"""QUESTION: {inputs['question']}
                GROUND TRUTH RESPONSE: {reference_outputs['answer']}
                STUDENT RESPONSE: {outputs['answer']}"""

    grade = await grader_llm.ainvoke([{"role": "system", "content": EVALUATION_PROMPT}, {"role": "user", "content": user}])
    is_correct = grade["is_correct"]
    return is_correct


# Target function
async def run_graph(inputs: dict) -> dict:
    """Run graph and track the trajectory it takes along with the final response."""
    result = await graph.ainvoke({
        "question": inputs["question"]
    })
    return {"answer": result["answer"].content}


# run evaluation
async def run_eval(
    dataset_name: str,
    experiment_prefix: Optional[str] = None,
) -> EvaluationResults:
    dataset = client.read_dataset(dataset_name=dataset_name)
    results = await aevaluate(
        run_graph,
        data=dataset,
        evaluators=[evaluate_agent],
        experiment_prefix=experiment_prefix,
    )
    return results


async def main():
    experiment_results = await run_eval(dataset_name=DEFAULT_DATASET_NAME,
                                        experiment_prefix="langgraph-blogs-qa-evals2")

#if __name__ == "__main__":
#    import asyncio
#    asyncio.run(main())

await main()

In [ ]:
from langchain import hub
from langchain.chat_models import init_chat_model
from langsmith import evaluate

# See the prompt: https://smith.langchain.com/hub/langchain-ai/pairwise-evaluation-2
prompt = hub.pull("langchain-ai/pairwise-evaluation-2")
model = init_chat_model("gpt-4o-mini")
chain = prompt | model

def ranked_preference(inputs: dict, outputs: list[dict]) -> list:
    # Assumes example inputs have a 'question' key and experiment
    # outputs have an 'answer' key.
    response = chain.invoke({
        "question": inputs["question"],
        "answer_a": outputs[0].get("answer", "N/A"),
        "answer_b": outputs[1].get("answer", "N/A"),
    })

    if response["Preference"] == 1:
        scores = [1, 0]
    elif response["Preference"] == 2:
        scores = [0, 1]
    else:
        scores = [0, 0]
    return scores

evaluate(
    ("langgraph-blogs-qa-evals-9aa78b29", "langgraph-blogs-qa-evals2-5031643f"),  # Replace with the names/IDs of your experiments
    evaluators=[ranked_preference],
    randomize_order=True,
    max_concurrency=4,
)